In [1]:
from utils.bigquery_tools import buscar_coches_bq 
import pandas as pd
import traceback
import logging

## paso a paso


In [2]:

filtros_test = {
    "transmision_preferida": "ambos", # No filtra
    "estetica_min": 5.0,
    "premium_min": 5.0,
    "singular_min": 5.0,
    "tipo_mecanica": ['GASOLINA', 'DIESEL'],
    "tipo_carroceria": ['TODOTERRENO', 'PICKUP', 'SUV', 'CROSSOVER'],
    "plazas_min": 2,
    "penalizar_puertas_bajas": False, # Para este escenario
    "modo_adquisicion_recomendado": "Contado", # O directamente poner precio_maximo
    "precio_max_contado_recomendado": 20000.0,
}

pesos_test = { # Ejemplo de pesos normalizados (asegúrate que los tuyos sumen ~1)
    "estetica": 0.027, 
     "premium": 0.135, 
     "singular": 0.135,
     "altura_libre_suelo": 0.216, 
     "batalla": 0.0, # Batalla sin peso en este ejemplo
     "indice_altura_interior": 0.0, # Índice sin peso en este ejemplo
     "ancho": 0.0, # Ancho sin peso en este ejemplo
     "traccion": 0.270, 
     "reductoras": 0.216
}
k_test = 7

In [ ]:
print("\n\n--- INICIANDO PRUEBA ESCENARIO 1: AVENTURERO EXTREMO (CONTADO) ---")
resultados_s1 = buscar_coches_bq(filtros=filtros_test, pesos=pesos_test, k=k_test)
print(f"Resultados encontrados: {len(resultados_s1)}")
print("Resultados:")
print(resultados_s1)

In [3]:
print("\n\n--- INICIANDO PRUEBA ESCENARIO 1 ---")
print(f"Filtros: {filtros_test}")
print(f"Pesos: {pesos_test}")

resultados_confort = buscar_coches_bq(
    filtros=filtros_test, 
    pesos=pesos_test, 
    k=k_test
)

print(f"\n--- RESULTADOS ESCENARIO CONFORT ({len(resultados_confort)} coches) ---")
if resultados_confort:
    df_confort = pd.DataFrame(resultados_confort)
    # Definir columnas_a_mostrar con las columnas que deseas mostrar
    columnas_a_mostrar = ['nombre', 'precio_compra_contado', 'tipo_carroceria', 'tipo_mecanica', 'premium', 'singular', 'estetica', 'plazas', 'puertas', 'ancho', 'altura_libre_suelo', 'batalla', 'indice_altura_interior', 'traccion', 'reductoras', 'score_total']
    columnas_existentes_confort = [col for col in columnas_a_mostrar if col in df_confort.columns]
    if columnas_existentes_confort:
        print(df_confort[columnas_existentes_confort].to_markdown(index=False))
    else:
        print("Resultados crudos:", resultados_confort)
else:
    print("No se encontraron resultados.")



--- INICIANDO PRUEBA ESCENARIO 1 ---
Filtros: {'transmision_preferida': 'ambos', 'estetica_min': 5.0, 'premium_min': 5.0, 'singular_min': 5.0, 'tipo_mecanica': ['GASOLINA', 'DIESEL'], 'tipo_carroceria': ['TODOTERRENO', 'PICKUP', 'SUV', 'CROSSOVER'], 'plazas_min': 2, 'penalizar_puertas_bajas': False, 'modo_adquisicion_recomendado': 'Contado', 'precio_max_contado_recomendado': 20000.0}
Pesos: {'estetica': 0.027, 'premium': 0.135, 'singular': 0.135, 'altura_libre_suelo': 0.216, 'batalla': 0.0, 'indice_altura_interior': 0.0, 'ancho': 0.0, 'traccion': 0.27, 'reductoras': 0.216}
--- 🧠 SQL Query (Paso a Paso) ---
 
    WITH ScaledData AS (
        SELECT
            *,
            
        COALESCE(SAFE_DIVIDE(COALESCE(estetica, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0)), 0) AS estetica_scaled,
        COALESCE(SAFE_DIVIDE(COALESCE(premium, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0)), 0) AS premium_scaled,
        COALESCE(SAFE_DIVIDE(COALESCE(singular, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0)), 0) AS singular_sca

In [ ]:
paso_paso_resultados_1 = buscar_coches_bq(filtros=filtros_test, pesos=pesos_test, k=k_test)
print(f"\n--- RESULTADOS ESCENARIO 1 ({len(paso_paso_resultados_1)} coches) ---")
print(paso_paso_resultados_1)

## Prueba 2

In [4]:
# --- Escenario "Profesional Confort Carretera" (Modo 1 Financiado) ---

filtros_test_confort = {
    # --- De Preferencias y Filtros Técnicos ---
    "transmision_preferida": "automático",
    "estetica_min": 5.0,  # valora_estetica='sí' (post-proc con apasionado='no')
    "premium_min": 1.0,   # apasionado_motor='no'
    "singular_min": 1.0,  # apasionado_motor='no'
    "tipo_mecanica": ["DIESEL", "PHEVD", "PHEVG", "GASOLINA"], # Eficientes para carretera
    "tipo_carroceria": ['3VOL', 'SUV', 'PICKUP'], # RAG para confort, viajes, profesional

    # --- De Lógica de Pasajeros ---
    "plazas_min": 2, # (X=0 + Z=1 + Conductor=1)
    "penalizar_puertas_bajas": False, # Frecuencia ocasional, X=0

    # --- De Lógica Económica (Modo 1 Financiado) ---
    "modo_adquisicion_recomendado": "Financiado",
    "precio_max_contado_recomendado": None,
    "cuota_max_calculada": 458.33, 
    
    # Campos de Modo 2 no aplican aquí
    "modo": None, 
    "submodo": None,
    "pago_contado": None,
    "cuota_max": None,
}

# Pesos simulados:
# Perfil: Estética=SÍ (crudo 5), Premium=NO (1), Singular=NO (1)
# Aventura NINGUNA (altura_libre=0, traccion=0, reductoras=0)
# Usuario NO Alto (batalla=0.5, indice_altura=0.5)
# NO Prioriza Ancho (ancho=0.5)
# Suma crudos: 5+1+1+0+0+0+0.5+0.5+0.5 = 8.5
pesos_test_confort = { 
    "estetica":             5.0 / 8.5,  # ~0.588 (Principal factor diferenciador)
    "premium":              1.0 / 8.5,  # ~0.118
    "singular":             1.0 / 8.5,  # ~0.118
    "altura_libre_suelo":   0.0 / 8.5,  # 0.0
    "traccion":             0.0 / 8.5,  # 0.0
    "reductoras":           0.0 / 8.5,  # 0.0
    "batalla":              0.5 / 8.5,  # ~0.059 (Bajo peso, confort en carretera podría valorarlo un poco)
    "indice_altura_interior": 0.5 / 8.5,  # ~0.059 (Bajo peso)
    "ancho":                0.5 / 8.5,  # ~0.059 (Bajo peso)
}
# print(f"Suma pesos confort: {sum(pesos_test_confort.values())}")

k_test_confort = 5 



In [5]:

print("\n\n--- INICIANDO PRUEBA ESCENARIO: PROFESIONAL CONFORT CARRETERA ---")
print(f"Filtros: {filtros_test_confort}")
print(f"Pesos: {pesos_test_confort}")

resultados_confort = buscar_coches_bq(
    filtros=filtros_test_confort, 
    pesos=pesos_test_confort, 
    k=k_test_confort
)

print(f"\n--- RESULTADOS ESCENARIO CONFORT ({len(resultados_confort)} coches) ---")
if resultados_confort:
    df_confort = pd.DataFrame(resultados_confort)
    # Definir columnas_a_mostrar con las columnas que deseas mostrar
    columnas_a_mostrar = ['nombre', 'precio_compra_contado', 'tipo_carroceria', 'tipo_mecanica', 'premium', 'singular', 'estetica', 'plazas', 'puertas', 'ancho', 'altura_libre_suelo', 'batalla', 'indice_altura_interior', 'traccion', 'reductoras', 'score_total']
    columnas_existentes_confort = [col for col in columnas_a_mostrar if col in df_confort.columns]
    if columnas_existentes_confort:
        print(df_confort[columnas_existentes_confort].to_markdown(index=False))
    else:
        print("Resultados crudos:", resultados_confort)
else:
    print("No se encontraron resultados.")



--- INICIANDO PRUEBA ESCENARIO: PROFESIONAL CONFORT CARRETERA ---
Filtros: {'transmision_preferida': 'automático', 'estetica_min': 5.0, 'premium_min': 1.0, 'singular_min': 1.0, 'tipo_mecanica': ['DIESEL', 'PHEVD', 'PHEVG', 'GASOLINA'], 'tipo_carroceria': ['3VOL', 'SUV', 'PICKUP'], 'plazas_min': 2, 'penalizar_puertas_bajas': False, 'modo_adquisicion_recomendado': 'Financiado', 'precio_max_contado_recomendado': None, 'cuota_max_calculada': 458.33, 'modo': None, 'submodo': None, 'pago_contado': None, 'cuota_max': None}
Pesos: {'estetica': 0.5882352941176471, 'premium': 0.11764705882352941, 'singular': 0.11764705882352941, 'altura_libre_suelo': 0.0, 'traccion': 0.0, 'reductoras': 0.0, 'batalla': 0.058823529411764705, 'indice_altura_interior': 0.058823529411764705, 'ancho': 0.058823529411764705}
--- 🧠 SQL Query (Paso a Paso) ---
 
    WITH ScaledData AS (
        SELECT
            *,
            
        COALESCE(SAFE_DIVIDE(COALESCE(estetica, 1.0) - 1.0, NULLIF(10.0 - 1.0, 0)), 0) AS es